# 커스텀 컨테이너 타입은 collection.abc의 클래스를 상속받게 만들자

Python 프로그래밍의 대부분은 데이터를 담은 클래스들을 정의하고 이 객체들이 연계되는 방법을 명시하는 일이다. 모든 파이썬 클래스는 일종의 컨테이너로, 속성과 기능을 함께 캡슐화한다. 파이썬은 데이터 관리용 내장 컨테이너 타입도 제공한다.

시퀀스처럼 쓰임새가 간단한 클래스를 설계할 때는 파이썬의 내장 list 타입에서 상속받으려고 하는 게 당연하다.

예를 들어 멤버의 빈도를 세는 메서드를 추가로 갖춘 커스텀 리스트 타입을 생성해보자.

In [1]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
        
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

list에서 상속방아 서브클래스를 만들었으므로 list의 표준 기능을 모두 갖춰서 Python 프로그래머에게 익숙한 시맨틱(sementic)을 유지한다. 그리고 추가한 메서드로 필요한 커스텀 동작을 더할 수 있다.

In [2]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


이제 list의 서브클래스는 아니지만 인덱스로 접근할 수 있게 해서 list처럼 보이는 객체를 제공하고 싶다 해보자.

예를 들어 바이너리 트리 클래스에 시퀀스 시맨틱을 제공한다고 하자.

In [3]:
class BinaryNode(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

이 클래스가 시퀀스 타입처럼 동작하게 하려면 어떻게 해야 할까?

파이썬은 특별한 이름을 붙인 인스턴스 메서드로 컨테이너 동작을 구현한다.

In [4]:
bar = [1, 2, 3]
bar[0]

1

위와 같이 시퀀스 아이템을 인덱스로 접근함녀 다음과 같이 해석된다.

In [5]:
bar.__getitem__(0)

1

BinaryNode 클래스가 시퀀스처럼 동작하게 하려면 객체의 트릴를 깊이 우선으로 탐색하는 `__getitem__`을 구현하면 된다.

In [13]:
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        found = None
        if self.left:
            found, count = self.left._search(count, index)
        if not found and count == index:
            found = self
        else:
            count += 1
        if not found and self.right:
            found, count = self.right._search(count, index)
        return found, count
        # Returns (found, count)
        
    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range')
        return found.value

In [14]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6, right=IndexableNode(7))),
    right=IndexableNode(
        15, left=IndexableNode(11)))

In [15]:
print('LRR =', tree.left.right.right.value)
print('Index 0 =', tree[0])
print('Index 1 =', tree[1])
print('11 in the tree?', 11 in tree)
print('17 in the tree?', 17 in tree)
print('Tree is', list(tree))

LRR = 7
Index 0 = 2
Index 1 = 5
11 in the tree? True
17 in the tree? False
Tree is [2, 5, 6, 7, 10, 11, 15]


문제는 `__getitem__`을 구현하는 것만으로는 기대하는 시퀀스 시맨틱을 모두 제공하지 못한다.

In [16]:
len(tree)

TypeError: object of type 'IndexableNode' has no len()

내장 함수 len을 쓰려면 커스텀 시퀀스 타입에 맞게 구현한 `__len__`이라는 또 다른 특별한 메서드가 필요하다.

In [18]:
class SequenceNode(IndexableNode):
    def __len__(self):
        _, count = self._search(0, None)
        return count
    
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6, right=SequenceNode(7))),
    right=SequenceNode(
        15, left=SequenceNode(11))
)

print('Tree has %d nodes' % len(tree))


Tree has 7 nodes


시퀀스 타입에서 기대할 count와 index 메서드가 빠졌다. 커스텀 컨테이너 타입을 정의하는 일은 보기보다 어렵다.

이런 어려움을 피하기 위해 내장 `collections.abc`모듈은 각 컨테이너 타입에 필요한 일반적인 메서드를 모두 제공하는 추상 기반 클래스들을 정의한다. 이 추상 기반 클래스들에서 상속받아 서브클래스를 만들다가 필수 메서드를 놓치면, 이를 말해준다.

In [21]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

이전의 SequenceNode 처럼 추상 기반 클래스가 요구하는 메서드를 모두 구현하면 별도로 작업하지 않아도 클래스가 index와 count 같은 부가적인 메서드를 모두 제공한다.

In [22]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6, right=BetterNode(7))),
    right=BetterNode(
        15, left=BetterNode(11))
)

print('Index of 7 is', tree.index(7))
print('Count of 10 is', tree.count(10))

Index of 7 is 3
Count of 10 is 1


Set와 MutableMapping처럼 파이썬의 관례에 맞춰 구현해야하는 특별한 메서드가 많은 더 복잡한 타입을 정의할 때 이런 추상 기반 클래스를 사용하는 이점은 커진다.